##### Silver Speech and Golden Silence: Spoiler Detection Project

### Review-wise Balanced Random Forest with Metadata (extra Feature)

In [1]:
#Imports
import pandas as pd
import numpy as np
from tqdm import tqdm
from imblearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.ensemble import BalancedRandomForestClassifier

#### Load Data

In [2]:
#Load train data
train = pd.read_json('data/train_reduced.json')
train.drop(['user_id', 'title'], inplace = True, axis = 1)
train.reset_index(drop = True, inplace = True)

In [3]:
#Load validation data
val = pd.read_json('data/validation_reduced.json')
val.drop(['user_id', 'title'], inplace = True, axis = 1)
val.reset_index(drop = True, inplace = True)

In [4]:
#Load reviews as a whole for train and validation
X_tr = pd.read_csv('data/X_train.csv')
X_v = pd.read_csv('data/X_val.csv')
#Rename columns
X_tr.rename(columns = {"0": 'review_whole'}, inplace = True)
X_v.rename(columns = {"0": 'review_whole'}, inplace = True)
#Drop unnamed columns
X_tr.drop('Unnamed: 0', inplace = True, axis = 1)
X_v.drop('Unnamed: 0', inplace = True, axis = 1)
#Reset indices
X_tr.reset_index(drop = True, inplace = True)
X_v.reset_index(drop = True, inplace = True)

In [5]:
#Combine dataframes and reviews 
train_df = pd.concat([train, X_tr], axis = 1)
val_df = pd.concat([val, X_v], axis = 1)

In [6]:
#Save data for later use
train_df.to_json('data/train_only_needed_columns.json')
val_df.to_json('data/val_only_needed_columns.json')

In [7]:
#Since the data is imbalanced, we downsample the non-spoilers. This further reduces the sample to about 25,000.
def downsample_nonspoilers_reviewwise(df):
    df_majority = df[df['spoiler_dum'] == 0] #nonspoilers
    df_minority = df[df['spoiler_dum'] == 1] #spoilers
    N = 2*len(df_minority)
    
    # Downsample majority labels equals three times the number of samples in the minority class while keeping the original genre proportion
    df_majority_ = df_majority.groupby('genre', group_keys = False).apply(lambda x: x.sample(int(np.rint(N*len(x)/len(df))))).sample(frac = 1)

    # Concatenate the majority and minority dataframes
    sample = pd.concat([df_majority_, df_minority], axis = 0)
    
    sample.reset_index(inplace = True, drop = True)
    return sample

In [1]:
train_df = downsample_nonspoilers_reviewwise(train_df)

In [57]:
#Rescale genre and review length with standardscaler
train_df.genre = train_df.genre.astype('category').cat.codes
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
train_df[['genre', 'review_len']] = pd.DataFrame(scaler.fit_transform(train_df[['genre', 'review_len']]))

In [72]:
#Metadata
X_train_meta = train_df[['genre', 'review_len']]
#Review texts
X_train_text = train_df.review_whole
#Labels
y_train = train_df.spoiler_dum

In [60]:
#Rescale genre and review length in validation data
val_df.genre = val_df.genre.astype('category').cat.codes
val_df[['genre', 'review_len']] = scaler.transform(val_df[['genre', 'review_len']])

In [73]:
#Metadata
X_val_meta = val_df[['genre', 'review_len']]
#Review texts
X_val_text = val_df.review_whole
#Labels
y_val = val_df.spoiler_dum

#### Pipeline for Balanced Random Forest with Review Texts and Genre Metadata

First, we run a balanced random forest on the training review texts and then use the predicted spoiler probabilities as a new feature for a second balanced random forest which combines the probabilities and genre information.

In [81]:
#Function for model training and prediction
def run_model(pipeline, X_train, y_train, X_test, y_test):
    #Fit the model
    model = pipeline.fit(X_train, y_train)
    
    #Predict labels of test data
    y_pred = model.predict(X_test)
    
    #Print classification report and confusion matrix
    return print(classification_report(y_test, y_pred)), print(confusion_matrix(y_test, y_pred, normalize = 'true'))

In [82]:
#Build a pipeline for feature extraction with TF IDF and SGD
#Feature Selection: K-Best
feat = SelectKBest(score_func = chi2, k = 10000)

#TFIDF
tfidf = TfidfVectorizer(stop_words = 'english', ngram_range = (1,2), min_df = 100)

#Balanced Random Forest classifier
brf = BalancedRandomForestClassifier(n_estimators = 100, random_state = 42, verbose = 5, n_jobs = -1, class_weight = 'balanced', oob_score = True)

pipe = Pipeline([('TF-IDF', tfidf), ('select_kbest', feat), ('BRF', brf)])

In [85]:
#run the model review-wise (validation)
run_model(pipe, X_train_text.astype('str'), y_train, X_train_text.astype('str'), y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100
building tree 4 of 100
building tree 2 of 100
building tree 3 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 11 of 100
building tree 12 of 100
building tree 10 of 100


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.7min


building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 31 of 100building tree 30 of 100

building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54

[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  9.0min


building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 13.3min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    4.4s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    6.8s finished


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     81811
           1       1.00      1.00      1.00     62786

    accuracy                           1.00    144597
   macro avg       1.00      1.00      1.00    144597
weighted avg       1.00      1.00      1.00    144597

[[9.99205486e-01 7.94514185e-04]
 [1.91125410e-04 9.99808875e-01]]


(None, None)

In [86]:
#Function to get an array with the predicted probabilities for each class
def get_probabilities(pipeline, X_train, y_train, X_test):
    #Fit the model
    model = pipeline.fit(X_train, y_train)
    
    #Predict probabilities of test data predictions
    return model.predict_proba(X_test)

In [87]:
proba_train = get_probabilities(pipe, X_train_text.astype('str'), y_train, X_train_text.astype('str'))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 4 of 100
building tree 3 of 100
building tree 1 of 100
building tree 2 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.5min


building tree 13 of 100
building tree 14 of 100
building tree 16 of 100building tree 15 of 100

building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 40 of 100
building tree 39 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54

[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  7.8min


building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 12.2min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    4.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    6.6s finished


In [88]:
#run the model review-wise (validation)
run_model(pipe, X_train_text.astype('str'), y_train, X_val_text.astype('str'), y_val)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100
building tree 3 of 100building tree 4 of 100

building tree 2 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.4min


building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54

[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  7.6min


building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 11.9min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    1.2s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    7.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   11.2s finished


              precision    recall  f1-score   support

           0       0.99      0.63      0.77    235341
           1       0.16      0.94      0.28     18062

    accuracy                           0.65    253403
   macro avg       0.58      0.78      0.52    253403
weighted avg       0.93      0.65      0.73    253403

[[0.62505471 0.37494529]
 [0.05619533 0.94380467]]


(None, None)

In [89]:
proba_val = get_probabilities(pipe, X_train_text.astype('str'), y_train, X_val_text.astype('str'))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 1 of 100
building tree 3 of 100
building tree 4 of 100
building tree 2 of 100
building tree 5 of 100
building tree 6 of 100building tree 7 of 100

building tree 8 of 100
building tree 9 of 100
building tree 11 of 100
building tree 12 of 100
building tree 10 of 100


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:  1.4min


building tree 13 of 100
building tree 14 of 100
building tree 16 of 100
building tree 15 of 100
building tree 17 of 100
building tree 19 of 100
building tree 18 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 28 of 100
building tree 27 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54

[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  7.6min


building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71 of 100
building tree 72 of 100
building tree 73 of 100
building tree 74 of 100
building tree 75 of 100
building tree 76 of 100
building tree 77 of 100
building tree 78 of 100
building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed: 11.9min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    7.2s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:   11.2s finished


In [90]:
#Function to combine metadata and probability for a spoiler
def df_meta_proba(meta_df, spoiler_probabilities_as_df):
    '''Give in a dataframe and the spoiler probabilities as pd.DataFrame'''
    #Combine metadata and probabilities
    combined_df = pd.concat([meta_df, spoiler_probabilities_as_df], axis = 1)
    #Rename the probability feature
    combined_df.rename(columns = {0: 'pred_spoiler_proba'}, inplace = True)
    return combined_df

In [122]:
X_train_combined = df_meta_proba(X_train_meta, pd.DataFrame(proba_train[:, [1]]))
X_train_combined 

,genre,review_len,pred_spoiler_proba
0,1.414380,-0.196730,0.35
1,1.080955,-0.687267,0.23
2,-1.253019,3.436192,0.42
3,1.080955,-0.561866,0.29
4,0.414106,-0.901185,0.01
...,...,...,...
144592,1.080955,1.230621,0.90
144593,-0.919594,0.216353,0.89
144594,-0.919594,-0.517607,0.84
144595,-0.919594,-0.240989,0.87


In [124]:
X_val_combined = df_meta_proba(X_val_meta, pd.DataFrame(proba_val[:, [1]]))
X_val_combined 

,genre,review_len,pred_spoiler_proba
0,-0.919594,-0.952820,0.21
1,-0.919594,-0.938067,0.21
2,-0.919594,-0.882744,0.71
3,-0.586169,-0.923314,0.40
4,1.080955,-0.956509,0.08
...,...,...,...
253398,-0.919594,-0.823732,0.63
253399,-0.919594,-0.853238,0.86
253400,-0.919594,-0.934379,0.38
253401,-0.586169,-0.875367,0.69


In [125]:
#Model on genre and probabilities
brf.fit(X_train_combined, y_train)
y_pred = brf.predict(X_val_combined)
print(classification_report(y_val, y_pred))
print(confusion_matrix(y_val, y_pred, normalize = 'true'))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


building tree 2 of 100building tree 1 of 100

building tree 4 of 100
building tree 3 of 100
building tree 5 of 100
building tree 6 of 100
building tree 8 of 100
building tree 7 of 100
building tree 9 of 100
building tree 11 of 100
building tree 10 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.6s


building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100building tree 31 of 100

building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 53 of 100
building tree 52 of 100
building tree 54 of 100
building tree 55 of 100
building tree 57 of 100building tree 56 of 100

building tree 58

[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:    2.3s


building tree 71 of 100
building tree 72 of 100
building tree 74 of 100building tree 73 of 100

building tree 75 of 100
building tree 77 of 100
building tree 78 of 100building tree 76 of 100

building tree 79 of 100
building tree 80 of 100
building tree 81 of 100
building tree 82 of 100
building tree 83 of 100
building tree 84 of 100
building tree 85 of 100
building tree 86 of 100
building tree 87 of 100
building tree 88 of 100
building tree 89 of 100
building tree 90 of 100
building tree 91 of 100
building tree 92 of 100
building tree 93 of 100
building tree 94 of 100
building tree 95 of 100
building tree 96 of 100
building tree 97 of 100
building tree 98 of 100
building tree 99 of 100
building tree 100 of 100


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.3s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done  64 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    0.5s finished


              precision    recall  f1-score   support

           0       0.99      0.63      0.77    235341
           1       0.16      0.94      0.28     18062

    accuracy                           0.65    253403
   macro avg       0.58      0.78      0.52    253403
weighted avg       0.93      0.65      0.73    253403

[[0.62669063 0.37330937]
 [0.05713653 0.94286347]]
